<div class="frontmatter text-center">
<h1> Introduction to data science and programming</h1>
<h2>Lecture 23: Machine learning</h2>
<h3>IT University of Copenhagen, Fall 2023</h3>
<h3>Instructor: Michael Szell</h3>
</div>

# Distance calculations

Docs: https://docs.scipy.org/doc/scipy/reference/spatial.distance.html#module-scipy.spatial.distance

In [ ]:
from scipy.spatial import distance

In [ ]:
rattlesnake = [1,1,1,1,0]
boa_constrictor = [0,1,0,1,0]
dartfrog = [1,0,1,0,4]
aligator = [1,1,0,1,4]

In [ ]:
distance.euclidean(aligator, boa_constrictor)

# Underfitting vs. Overfitting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

This example demonstrates the problems of <B>underfitting</B> and <B>overfitting</B> and how we can use linear regression with polynomial features to approximate nonlinear functions. The plot shows the function that we want to approximate, which is a part of the cosine function. In addition, the samples from the real function and the approximations of different models are displayed. The models have polynomial features of different degrees. We can see that a linear function (polynomial with degree 1) is not sufficient to fit the training samples. This is called underfitting. A polynomial of degree 4 approximates the true function almost perfectly. However, for higher degrees the model will overfit the training data, i.e. it learns the noise of the training data. We evaluate quantitatively <B>overfitting / underfitting</B> by using cross-validation. We calculate the mean squared error (MSE) on the validation set, the higher, the less likely the model generalizes correctly from the training data.

In [ ]:
#Function that describe the data
def true_fun(X):
    return np.cos(1.5 * np.pi * X)

#sets a random seed
np.random.seed(0)

#number of points
n_samples = 30

#number of degrees to be tested
degrees = [1, 4, 15]

X = np.sort(np.random.rand(n_samples))
y = true_fun(X) + np.random.randn(n_samples) * 0.1

labels = ['Underfitting', 'Good model', 'Overfitting']
count = 0 

plt.figure(figsize=(14, 5))
for i in range(len(degrees)):
    ax = plt.subplot(1, len(degrees), i + 1)
    plt.setp(ax, xticks=(), yticks=())

    #generate the polinomial functions
    
    #Generate a new feature matrix consisting of all polynomial combinations of the features with degree less than 
    #or equal to the specified degree. For example, if an input sample is two dimensional and of the form [a, b], 
    #the degree-2 polynomial features are [1, a, b, a^2, ab, b^2].
    
    polynomial_features = PolynomialFeatures(degree=degrees[i],
                                             include_bias=False)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X[:, np.newaxis], y)

    #Evaluate the models using crossvalidation
    scores = cross_val_score(pipeline, X[:, np.newaxis], y,
                             scoring="neg_mean_squared_error", cv=10)

    #plot the functions
    X_test = np.linspace(0, 1, 100)
    plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Model")
    plt.plot(X_test, true_fun(X_test), label="True function")
    plt.scatter(X, y, edgecolor='b', s=20, label="Samples")
    plt.xlabel("x ({})".format(labels[count]))
    count+=1
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    plt.title("Degree {}\nMSE = {:.2e}(+/- {:.2e})".format(
        degrees[i], -scores.mean(), scores.std()))
plt.show()